https://digitalcommons.chapman.edu/cgi/viewcontent.cgi?article=1027&context=economics_articles

In [1]:
from edsl import QuestionNumerical, Scenario, QuestionMultipleChoice, Survey

s = Scenario({'value': {'John':10, 'Robin':9}, 'N':3, 'players': ['John', 'Robin']})

q_b = QuestionNumerical(question_name = "buyer", 
                      question_text = """
                      You are a buyer in a market for a commodity.
                      {% set player = scenario.players | random %}
                      Your name is {{ player }}.
                      Your value for the commodity is {{ scenario.value[player] }}.
                      You can call out a bid to other traders in the market. 
                      You would be happy to get the good at your value, but you'd like to get it for less.
                      Anything price you pay less than your value is pure profit.
                      What is your bid?
                      """)

q_s = QuestionMultipleChoice(question_name = """seller""", 
                      question_text = """
                      You are a seller in a market. 
                      Your value for the commodity is {{ scenario.value.Robin }}.
                      There are {{ N }} other sellers and {{ N }} other buyers.
                      You would be happy to sell the commodity at your value, but you'd like to get more for it.
                      Anything price you pay more than your value is pure profit.
                      There is one buyer offer for {{ buyer.answer }}
                      """, 
                    question_options = ["Accept an existing offer", "Ask for a different price"]
                          )
q_s_ask = QuestionNumerical(question_name = "seller_ask", 
                      question_text = """
                      {{ seller.question_text }}
                      You want to ask for a price. 
                      You chose {{ seller.answer }}.
                      What is your ask?
                      """)


results = (Survey([q_b, q_s,q_s_ask])
           .add_stop_rule('seller', '{{ seller.answer }} == "Accept an existing offer"')
           .by(s)
           .run(disable_remote_inference = True)
          )

In [2]:
results.select('buyer', 'seller', 'scenario.value', 'seller_ask', 'prompt.buyer_user_prompt')

,answer.buyer,answer.seller,scenario.value,answer.seller_ask,prompt.buyer_user_prompt
0,8.500000,Ask for a different price,"{'John': 10, 'Robin': 9}",9.500000,"You are a buyer in a market for a commodity. Your name is John. Your value for the commodity is 10. You can call out a bid to other traders in the market. You would be happy to get the good at your value, but you'd like to get it for less. Anything price you pay less than your value is pure profit. What is your bid? This question requires a numerical response in the form of an integer or decimal (e.g., -12, 0, 1, 2, 3.45, ...). Respond with just your number on a single line. If your response is equivalent to zero, report '0' After the answer, put a comment explaining your choice on the next line."


In [9]:
s.to_dict()

{'value': {'John': 10, 'Robin': 5},
 'edsl_version': '0.1.47.dev1',
 'edsl_class_name': 'Scenario'}

In [10]:
from edsl import QuestionFreeText
q = QuestionFreeText(
    question_name = "avg", 
    question_text = """
{% set numbers = [1,2,3,4,5, 150] %}
{% macro average(nums) %}
  {# Check if the list is not empty to avoid division by zero #}
  {% if nums|length > 0 %}
    {# Use the sum filter directly on the list #}
    {{ (nums|sum / nums|length)|float }}
  {% else %}
    {# Return 0 or some indication that the list was empty #}
    {{ 0 }}
  {% endif %}
{% endmacro %}
Is your favorite number less than {{ average(numbers) }}?
    """)
r = q.run(disable_remote_inference = True)

In [11]:
r.select('prompt.*')

,prompt.avg_user_prompt,prompt.avg_system_prompt
0,Is your favorite number less than 27.5 ?,nan
